In [ ]:
!nvidia-smi

Sat Jan  9 05:34:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install bert-for-tf2 >> /dev/null

In [ ]:
import os
import math
import datetime

# from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm

import pandas as pd
import numpy as np

import re

import tensorflow as tf
from tensorflow import keras
from  tensorflow.keras.callbacks import EarlyStopping

import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

import random

In [ ]:
cd "/content/drive/My Drive/Colab Notebooks NU"

/content/drive/.shortcut-targets-by-id/1FqifPUlm6cgaaWF0H2AQqipYJL1XcCct/Colab Notebooks


In [ ]:
import bert_thesis_experiments.utils as utils
import bert_thesis_experiments.my_models as my_models

In [ ]:
seqfile = 'DeepDom_Code/DeepDom-master/processed_seq.txt'; #file name of the processed sequence data (output from dataprocess.pl)
labelfile= 'DeepDom_Code/DeepDom-master/processed_label.txt'; #file name of the processed label data (output from dataprocess.pl)

In [ ]:
(ids,seqs) = utils.process_inputseqs(seqfile)
(ids,labels) = utils.process_inputlabels(labelfile)

In [ ]:
# num_of_seq = 20000
# rawdata = list(zip(seqs[:num_of_seq],labels[:num_of_seq]))
# random.shuffle(rawdata)

rawdata = list(zip(seqs,labels))
random.shuffle(rawdata)

In [ ]:
tokenizer = FullTokenizer(vocab_file=os.path.join("bert_thesis_experiments/working/", "protein_seq_words_uniref0.5_vsz30k_mfq2-vocab.txt"))

In [ ]:
seq_i = 51
print(rawdata[seq_i][0])
print("Number of tokens in seq", len(tokenizer.tokenize(rawdata[seq_i][0])))
print(tokenizer.tokenize(rawdata[seq_i][0]))

MSSPILNTDDNNCVRVALRVRPLSKKEEAERSMEVVKYVDGEPQVIMGDNNQTFTFDYVFNGKSRQQEIFDDCVANLVDCLFEGYNSTILAYGQTGSGKTFTMGTTSTIGIPTEELGVIPRVIDFIYDKIDRKKDTHQLVLKVSFLELYNEEIRDMLNPYPTAGGLPIREKSNGEVYIPGLVEQIVRSRQQMEEALIRGS
Number of tokens in seq 72
['mss', '##pi', '##l', '##ntdd', '##nnc', '##vr', '##v', '##alrv', '##rpl', '##skk', '##eeae', '##rsm', '##evv', '##kyv', '##dgep', '##qvi', '##mgd', '##nnq', '##tf', '##tf', '##dy', '##vf', '##ngk', '##srqq', '##eifd', '##dc', '##v', '##anlv', '##dc', '##l', '##feg', '##ynst', '##ilay', '##gq', '##tgsgkt', '##ftm', '##gt', '##ts', '##tigi', '##pt', '##eelgv', '##ipr', '##vi', '##df', '##iy', '##dk', '##idr', '##kkdt', '##hq', '##l', '##vlkv', '##sfl', '##elyn', '##eeir', '##dm', '##l', '##npy', '##pt', '##aggl', '##pir', '##eksn', '##ge', '##vy', '##ipgl', '##ve', '##qiv', '##rsr', '##qqm', '##eeal', '##ir', '##g', '##s']


In [ ]:
tokens = tokenizer.tokenize(rawdata[seq_i][0])
tokens = ["[CLS]"] + tokens + ["[SEP]"]
token_ids = tokenizer.convert_tokens_to_ids(tokens)
padded_token_ids = token_ids + [0] * (200 - len(token_ids))
print(len(padded_token_ids), padded_token_ids, sep='\n')

200
[2, 2122, 238, 42, 22392, 3456, 278, 31, 16893, 870, 410, 4108, 2283, 510, 1869, 16997, 1317, 2128, 1962, 292, 292, 315, 333, 4169, 16746, 15881, 490, 31, 5881, 490, 42, 17731, 19792, 29044, 322, 18138, 3321, 252, 307, 16216, 219, 29716, 8954, 228, 281, 332, 259, 6728, 23277, 284, 42, 17635, 422, 25384, 10548, 399, 42, 2129, 219, 2879, 12769, 19357, 330, 370, 7512, 294, 16270, 803, 2679, 2044, 267, 30, 43, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# import importlib
# importlib.reload(my_models)

<module 'bert_thesis_experiments.my_models' from '/content/drive/My Drive/Colab Notebooks/bert_thesis_experiments/my_models.py'>

In [ ]:
max_seq_len = utils.find_max_seq_len(rawdata, tokenizer)

print("\nMax Seq Length:", max_seq_len)



Max Seq Length: 170


In [ ]:
print(rawdata[seq_i][1])
print(rawdata[900][1])
print(rawdata[500][1])

11111111111111111111111111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001111
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001111111111111111111111111111111111111111100------------------------------------------------------


In [ ]:

label_array_1 = list(map(int, rawdata[500][1].replace('-', '0')))
print(label_array_1)
print(len(label_array_1))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
200


In [ ]:
inputX = [utils.convert_seq_ids(i[0], tokenizer, 200) for i in rawdata]
# inputY = [utils.convertlabels_to_categorical(i[1]) for i in rawdata]
inputY = [utils.convertlabels_to_binary(i[1]) for i in rawdata]

In [ ]:
print("Max length of sequence:", len(inputX[seq_i]))
print("No of sequences", len(inputX))
print(inputX[seq_i])

Max length of sequence: 200
No of sequences 31400
[2, 2122, 238, 42, 22392, 3456, 278, 31, 16893, 870, 410, 4108, 2283, 510, 1869, 16997, 1317, 2128, 1962, 292, 292, 315, 333, 4169, 16746, 15881, 490, 31, 5881, 490, 42, 17731, 19792, 29044, 322, 18138, 3321, 252, 307, 16216, 219, 29716, 8954, 228, 281, 332, 259, 6728, 23277, 284, 42, 17635, 422, 25384, 10548, 399, 42, 2129, 219, 2879, 12769, 19357, 330, 370, 7512, 294, 16270, 803, 2679, 2044, 267, 30, 43, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
data=list(zip(inputX,inputY))
random.seed(4)
random.shuffle(data)
random.shuffle(data)
    
train_num=int(len(inputX)*0.9)
train=data[0:train_num]
val=data[train_num:]

trainX=np.array([i[0] for i in train])
trainY=np.array([i[1] for i in train])
# xx=np.dstack(trainX)
# xx=np.rollaxis(xx,-1)

print(trainX.shape)   
print(trainY.shape) 

# yy=np.dstack(trainY)
# yy=np.rollaxis(yy,-1)

# print(yy.shape)
    
valX=np.array([i[0] for i in val])
valY=np.array([i[1] for i in val])
# valX=np.dstack(valX)
# valX=np.rollaxis(valX,-1)

# valY=np.dstack(valY)
# valY=np.rollaxis(valY,-1)

vv=(valX,valY)

print("\n---Validation Data---")
print(valX.shape)
print(valY.shape)

(28260, 200)
(28260, 200)

---Validation Data---
(3140, 200)
(3140, 200)


In [ ]:
bert_model_name = "bert_H768_A12_uniref50_vocab30k_steps30k"

bert_ckpt_dir = os.path.join("bert_thesis_experiments/working/", bert_model_name)
bert_ckpt_file = os.path.join(bert_ckpt_dir, "model.ckpt-10000")
bert_config_file = os.path.join(bert_ckpt_dir, "bert_config.json")

In [ ]:
def create_model_bilstm(max_seq_len, bert_ckpt_file):

  with tf.io.gfile.GFile(bert_config_file, "r") as reader:
      bc = StockBertConfig.from_json_string(reader.read())
      bert_params = map_stock_config_to_params(bc)
      bert_params.adapter_size = None
      bert = BertModelLayer.from_params(bert_params, name="bert")
        
  input_ids = keras.layers.Input(shape=(max_seq_len, ), dtype='int32', name="input_ids")
  bert_output = bert(input_ids)

  print("bert shape", bert_output.shape)

  classes = 200

  x1 = keras.layers.Bidirectional(keras.layers.LSTM(100,return_sequences=True),merge_mode='sum')(bert_output)
#   x4 = keras.layers.Bidirectional(keras.layers.LSTM(3,return_sequences=True),merge_mode='sum')(bert_output)
    
  x2 = keras.layers.Bidirectional(keras.layers.LSTM(100,return_sequences=True),merge_mode='sum')(x1)

  x3 = keras.layers.Bidirectional(keras.layers.LSTM(100,return_sequences=True),merge_mode='sum')(x2)
  x4 = keras.layers.Bidirectional(keras.layers.LSTM(3,return_sequences=True),merge_mode='sum')(x3)

  output = keras.layers.Activation('softmax')(x4)


#   cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
#   cls_out = keras.layers.Dropout(0.5)(cls_out)
#   logits = keras.layers.Dense(units=512, activation="tanh")(cls_out)
#   logits = keras.layers.Dropout(0.5)(logits)
#   logits = keras.layers.Dense(units=classes, activation="sigmoid")(logits)

  model = keras.Model(inputs=input_ids, outputs=output)
  model.build(input_shape=(None, max_seq_len))

  load_stock_weights(bert, bert_ckpt_file)
        
  return model

In [ ]:
def create_model(max_seq_len, bert_ckpt_file, bert_config_file):

    with tf.io.gfile.GFile(bert_config_file, "r") as reader:
        bc = StockBertConfig.from_json_string(reader.read())
        bert_params = map_stock_config_to_params(bc)
        bert_params.adapter_size = None
        bert = BertModelLayer.from_params(bert_params, name="bert")
    
    input_ids = keras.layers.Input(
        shape=(max_seq_len, ),
        dtype='int32',
        name="input_ids"
    )
    bert_output = bert(input_ids)
    
    print("bert shape", bert_output.shape)

    classes = 200

    # cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
    cls_out = keras.layers.Flatten()(bert_output)
    # cls_out = keras.layers.Dropout(0.5)(cls_out)
    # logits = keras.layers.Dense(units=512, activation="tanh")(cls_out)
    # logits = keras.layers.Dropout(0.5)(logits)
    logits = keras.layers.Dense(
        units=classes,
        activation="sigmoid"
    )(cls_out)

    model = keras.Model(inputs=input_ids, outputs=logits)


    load_stock_weights(bert, bert_ckpt_file)
    model.build(input_shape=(None, max_seq_len))

    return model

In [ ]:
def create_model_fcnn(max_seq_len, bert_ckpt_file, bert_config_file):

    with tf.io.gfile.GFile(bert_config_file, "r") as reader:
        bc = StockBertConfig.from_json_string(reader.read())
        bert_params = map_stock_config_to_params(bc)
        bert_params.adapter_size = None
        bert = BertModelLayer.from_params(bert_params, name="bert")
    
    input_ids = keras.layers.Input(
        shape=(max_seq_len, ),
        dtype='int32',
        name="input_ids"
    )
    bert_output = bert(input_ids)
    
    print("bert shape", bert_output.shape)

    classes = 200

    # cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
    cls_out = keras.layers.Flatten()(bert_output)
    logits = keras.layers.Dense(units=1500, activation="relu")(cls_out)
    logits = keras.layers.Dropout(0.5)(logits)
    for i in range(0,3):
        logits = keras.layers.Dense(units=1500)(logits)
        logits = keras.layers.Activation("relu")(logits)
    logits = keras.layers.Dropout(0.5)(logits)
    logits = keras.layers.Dense(
        units=classes,
        activation="sigmoid"
    )(logits)

    model = keras.Model(inputs=input_ids, outputs=logits, name="bert_fcnn")


    load_stock_weights(bert, bert_ckpt_file)
    model.build(input_shape=(None, max_seq_len))

    return model

In [ ]:
# model = my_models.create_model_bilstm(200, bert_ckpt_file, bert_config_file)
model = create_model_fcnn(200, bert_ckpt_file, bert_config_file)

bert shape (None, 200, 768)
Done loading 196 BERT weights from: bert_thesis_experiments/working/bert_H768_A12_uniref50_vocab30k_steps30k/model.ckpt-10000 into <bert.model.BertModelLayer object at 0x7fc213a73b00> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/LayerNorm/beta/adam_m
	bert/embeddings/LayerNorm/beta/adam_v
	bert/embeddings/LayerNorm/gamma/adam_m
	bert/embeddings/LayerNorm/gamma/adam_v
	bert/embeddings/position_embeddings/adam_m
	bert/embeddings/position_embeddings/adam_v
	bert/embeddings/token_type_embeddings
	bert/embeddings/token_type_embeddings/adam_m
	bert/embeddings/token_type_embeddings/adam_v
	bert/embeddings/word_embeddings/adam_m
	bert/embeddings/word_embeddings/adam_v
	bert/encoder/layer_0/attention/output/LayerNorm/beta/adam_m
	bert/encoder/layer_0/attention/output/LayerNorm/beta/adam_v
	bert/encoder/layer_0/attention/output/LayerNorm/gamma/adam_m


In [ ]:
model.summary()

Model: "bert_fcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 200)]             0         
_________________________________________________________________
bert (BertModelLayer)        (None, 200, 768)          108890112 
_________________________________________________________________
flatten_4 (Flatten)          (None, 153600)            0         
_________________________________________________________________
dense_36 (Dense)             (None, 1500)              230401500 
_________________________________________________________________
dropout_11 (Dropout)         (None, 1500)              0         
_________________________________________________________________
dense_37 (Dense)             (None, 1500)              2251500   
_________________________________________________________________
activation_15 (Activation)   (None, 1500)              0 

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False), optimizer=keras.optimizers.Adam(learning_rate=2e-5), 
              metrics=[keras.metrics.BinaryAccuracy(threshold=0.5)])
# model.compile(loss='binary_crossentropy',optimizer=keras.optimizers.Adam(learning_rate=2e-5), metrics=['accuracy'])

fitHistory_batch = model.fit(x=trainX,y=trainY, batch_size=16, validation_data=vv, epochs=5, callbacks=[early_stopping])
# fitHistory_batch = model.fit(x=trainX, y=yy, batch_size=32, validation_data=vv, epochs=3, callbacks=[early_stopping])

Epoch 1/5
1767/1767 [==============================] - 1805s 1s/step - loss: 0.5248 - binary_accuracy: 0.7883 - val_loss: 0.5251 - val_binary_accuracy: 0.8194
Epoch 2/5
1767/1767 [==============================] - 1786s 1s/step - loss: 0.4803 - binary_accuracy: 0.8191 - val_loss: 0.5060 - val_binary_accuracy: 0.8198
Epoch 3/5
1767/1767 [==============================] - 1783s 1s/step - loss: 0.4724 - binary_accuracy: 0.8207 - val_loss: 0.4966 - val_binary_accuracy: 0.8212
Epoch 4/5
1767/1767 [==============================] - 1783s 1s/step - loss: 0.4681 - binary_accuracy: 0.8212 - val_loss: 0.5009 - val_binary_accuracy: 0.8223
Epoch 5/5
1767/1767 [==============================] - 1784s 1s/step - loss: 0.4665 - binary_accuracy: 0.8217 - val_loss: 0.4876 - val_binary_accuracy: 0.8219


In [ ]:
model.save("bert-768-ConDo_fcnn_flatten_stp30k_voc30k_epoch5_bsz16")

INFO:tensorflow:Assets written to: bert-768-ConDo_fcnn_flatten_stp30k_voc30k_epoch5_bsz16/assets


In [ ]:
!cp bert_thesis_experiments/working/ibert_config.json bert_thesis_experiments/working/bert_H512_A8_uniref50_vocab80k_steps10k